In [1]:
import os
import pickle

from gensim.models import KeyedVectors

import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier

from data import preprocess_data, vectorize_data, load_dataset
from evaluation import analysis, evaluate_models_with_data
from w2v_adapter import Word2VecAdapter

import advanced_processor_chain_factory
import simple_processor_chain_factory

In [2]:
dataset = load_dataset()
DEBUG = False

# Inspection of Pre-Processing Approaches

In [ ]:
models = {'logistic regression' : LogisticRegression(class_weight = 'balanced', n_jobs=-1),
          'svm' : svm.LinearSVC(),
          'knn' : KNeighborsClassifier(n_neighbors=8, n_jobs=-1)
         }

## Without Pre-Process

In [ ]:
evaluate_models_with_data(models, *vectorize_data(*preprocess_data(dataset, debug=DEBUG), CountVectorizer(max_features=2000)))

## Simple Pre-Process

In [ ]:
evaluate_models_with_data(models,
                          *vectorize_data(
                              *preprocess_data(dataset, processor_chain=simple_processor_chain_factory.create(), debug=DEBUG),
                              CountVectorizer(max_features=2000)))

## Pre-Process with Stemmimg

In [ ]:
evaluate_models_with_data(models,
                          *vectorize_data(
                              *preprocess_data(dataset, processor_chain=advanced_processor_chain_factory.create('stem'),
                                               debug=DEBUG),
                              CountVectorizer(max_features=2000)))

## Pre-Process with Lemmitization

In [3]:
X, Y = preprocess_data(dataset, processor_chain=advanced_processor_chain_factory.create('lem'), debug=DEBUG)
del dataset

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
X_train_bow, X_test_bow, Y_train_bow, Y_test_bow = vectorize_data(X, Y,CountVectorizer(max_features=2000))
evaluate_models_with_data(models, X_train_bow, X_test_bow, Y_train_bow, Y_test_bow)

In [ ]:
del models

# Compare W2V and BoW with Their Best Tuned Hyper-parameters

In [ ]:
kfold = StratifiedKFold(n_splits=5)
general_grid_params = {'verbose' : 1, 'cv' : kfold, 'n_jobs' : -1, 'scoring' : 'f1'}

logistic_grid = {
    'penalty':['l2'],
    'C':[1, 300, 500, 700, 900, 2000],
    'class_weight':['balanced'],
    'solver':['saga'],
    'n_jobs':[-1],
    'max_iter':[1000],
}

svc_grid = {
    'kernel' : ['linear', 'rbf'],
    'C':[0.1, 1, 300, 500, 700],
}

knn_grid = {
    'n_neighbors' : [i for i in range(1,24,2)],
    'n_jobs' : [-1]
}

## BoW

### Logistic Regression

In [ ]:
bow_log = LogisticRegression()
bow_log = GridSearchCV(estimator=bow_log, param_grid=logistic_grid, **general_grid_params)
bow_log.fit(X_train_bow, Y_train_bow)
print(f'Best Score: {bow_log.best_score_}')
print(f'Best Params: {bow_log.best_params_}')

In [ ]:
analysis(Y_test_bow, bow_log.predict(X_test_bow))

### SVM

In [ ]:
bow_svm = svm.SVC()
bow_svm = GridSearchCV(estimator=bow_svm, param_grid=svc_grid, **general_grid_params)
bow_svm.fit(X_train_bow, Y_train_bow)
print(f'Best Score: {bow_svm.best_score_}')
print(f'Best Params: {bow_svm.best_params_}')

In [ ]:
analysis(Y_test_bow, bow_svm.predict(X_test_bow))

### KNN

In [ ]:
bow_knn = KNeighborsClassifier()
bow_knn = GridSearchCV(estimator=bow_knn, param_grid=knn_grid, **general_grid_params)
bow_knn.fit(X_train_bow, Y_train_bow)
print(f'Best Score: {bow_svm.best_score_}')
print(f'Best Params: {bow_svm.best_params_}')

In [ ]:
analysis(Y_test_bow, bow_svm.predict(X_test_bow))


## W2V

In [4]:
X_train_w2v, X_test_w2v, Y_train_w2v, Y_test_w2v = train_test_split(X, Y)

if os.path.isfile('w2v.kv'):
    vectorizer = Word2VecAdapter(KeyedVectors.load('vectors.kv'))
else:
    vectorizer = Word2VecAdapter()
    vectorizer.wv.save('w2v.kv')

X_train_w2v = vectorizer.fit_transform(X_train_w2v)
X_test_w2v = vectorizer.transform(X_test_w2v)


Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Logistic Regression

In [ ]:
conf = {'C': 1, 'class_weight': 'balanced', 'max_iter': 1000, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'saga'}
w2v_log = LogisticRegression(**conf)
w2v_log.fit(X_train_w2v, Y_train_w2v)
analysis(Y_test_w2v, w2v_log.predict(X_test_w2v))

In [ ]:
w2v_log = LogisticRegression()
w2v_log = GridSearchCV(estimator=w2v_log, param_grid=logistic_grid, **general_grid_params)
w2v_log.fit(X_train_w2v, Y_train_w2v)
print(f'Best Score: {w2v_log.best_score_}')
print(f'Best Params: {w2v_log.best_params_}')

In [ ]:
analysis(Y_test_w2v, w2v_log.predict(X_test_w2v))

### SVM

In [ ]:
w2v_svm = svm.SVC()
w2v_svm = GridSearchCV(estimator=w2v_svm, param_grid=svc_grid, **general_grid_params)
w2v_svm.fit(X_train_w2v, Y_train_w2v)
print(f'Best Score: {w2v_svm.best_score_}')
print(f'Best Params: {w2v_svm.best_params_}')

In [ ]:
analysis(Y_test_w2v, w2v_svm.predict(X_test_w2v))

### KNN

In [ ]:
w2v_knn = KNeighborsClassifier()
w2v_knn = GridSearchCV(estimator=w2v_knn, param_grid=knn_grid, **general_grid_params)
w2v_knn.fit(X_train_w2v, Y_train_w2v)
print(f'Best Score: {w2v_svm.best_score_}')
print(f'Best Params: {w2v_svm.best_params_}')

In [ ]:
analysis(Y_test_w2v, w2v_svm.predict(X_test_w2v))

## Comparison

In [ ]:
summary = {
    'LR': {'BoW': bow_log, 'W2V': w2v_log},
    'SVM' : {'BoW': bow_svm, 'W2V': w2v_svm},
    'KNN': {'BoW': bow_knn, 'W2V': w2v_knn},
  }

for name, values in summary.items():
    print(f'For classifier {name}, best BoW score is {values["BoW"].best_score_}, whereas best W2V score is {values["W2V"].best_score_}')
    best_model = "BoW" if values["BoW"].best_score_ > values["W2V"].best_score_ else "W2V"
    print(f'So {best_model} is better with parameters {values[best_model].best_params_}')
    filename = name + '.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(values[best_model], f)

del summary, bow_log, w2v_log, bow_svm, w2v_svm, bow_knn, w2v_knn

# MLP

In [ ]:
mlp_grid = {
    #
    #(1000, 250), (500, 250, 250),(1000, 500,250)
    'hidden_layer_sizes':[(500, 250)],
    #, 'relu'
    'activation':['tanh']
}
def eval_mlp(X_train, X_test, Y_train, Y_test):
    best_f1 = -1
    best_model = None
    for sizes in mlp_grid['hidden_layer_sizes']:
        for act in mlp_grid['activation']:
            m = MLPClassifier(hidden_layer_sizes=sizes, activation=act, solver='sgd', alpha=1,
                                    learning_rate='adaptive', max_iter=10)
            m.fit(X_train, Y_train)
            print(f'Model config: hidden_layer_sizes={sizes}, activation={act}')
            f1 = analysis(Y_test, m.predict(X_test))
            if f1 > best_f1:
                best_model = m
                best_f1 = f1
    return best_f1, best_model

## W2V

In [ ]:
w2v_f1, w2v_mlp = eval_mlp(X_train_w2v, X_test_w2v, Y_train_w2v, Y_test_w2v)

In [ ]:
del X_train_w2v, X_test_w2v, Y_train_w2v, Y_test_w2v

## BoW

In [ ]:
bow_f1, bow_mlp = eval_mlp(X_train_bow, X_test_bow, Y_train_bow, Y_test_bow)

In [ ]:
del X_train_bow, X_test_bow, Y_train_bow, Y_test_bow

## TD-IDF

In [ ]:
X_train_idf, X_test_idf, Y_train_idf, Y_test_idf = vectorize_data(X, Y, TfidfVectorizer(max_features=2000))

In [ ]:
tf_idf_f1, tf_idf_mlp = eval_mlp(X_train_idf, X_test_idf, Y_train_idf, Y_test_idf)

In [ ]:
del X_train_idf, X_test_idf, Y_train_idf, Y_test_idf

## Comparison

In [ ]:
print('Best scores:')
print(f'W2V: {w2v_f1} with params: {w2v_mlp.get_params()}')
print(f'BoW: {bow_f1} with params: {bow_mlp.get_params()}')
print(f'Tf-Idf: {tf_idf_f1} with params: {tf_idf_mlp.get_params()}')

idx = np.argmax([w2v_f1, bow_f1, tf_idf_f1])
best_mlp = [w2v_mlp, bow_mlp, tf_idf_mlp][idx]

with open('best.pkl', 'wb') as f:
    pickle.dump(best_mlp, f)